In [1]:
### Environment setup
from pystac_client import Client
import planetary_computer as pc
import os

# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)
env_vars = !cat /content/.env

for var in env_vars:
    key, value = var.split(' = ')
    os.environ[key] = value

In [2]:
import pandas as pd
import fsspec
import numpy as np
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt

data_srcs = ['itv', 'ana', 'usgs']
src = data_srcs[0]
container = f'{src}-data'

storage_options={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY']}
fs = fsspec.filesystem('az', account_name=storage_options['account_name'], account_key=storage_options['account_key'])

In [21]:
total_tifs = []
total_csvs = []
for src in data_srcs:
    container = f'{src}-data'
    station_url = f'az://{container}/{src}_station_metadata.csv'
    df = pd.read_csv(station_url, storage_options=storage_options)
    folder = f'{container}/stations'
    station_names = [str(f).zfill(8) for f in df.site_no]
    #iterate on stations
    tif_count = []
    for station_folder in station_names:
        s = f'{folder}/{station_folder}'
        s_list = fs.ls(s)
        if s_list is not None:
            total = sum([s.count('.tif') for s in s_list])
            total_csv = sum([s.count('.csv') for s in s_list])
            tif_count.append({'site_no':station_folder, 'tif_count':total, 'csv_count':total_csv})
    tif_count = pd.DataFrame(tif_count)
    print(f'{src} data has {np.sum(tif_count.tif_count)/2} tifs and {np.sum(tif_count.csv_count)} csvs')
    total_tifs.append(tif_count)
total_tifs = pd.concat(total_tifs)

itv data has 198.0 tifs and 16 csvs
ana data has 143.0 tifs and 64 csvs
usgs data has 6315.0 tifs and 69 csvs


In [18]:
sdf = pd.read_csv(f'az://{s_list[0]}',storage_options=storage_options)
date_range = f"{sdf['Date-Time'].iloc[0]}-{sdf['Date-Time'].iloc[-1]}"

In [19]:
date_range

'07/01/2015 01:00-04/26/2017 13:00'

In [8]:
img_df

site_no                                          site_name  Latitude  \
0   1632900                    Smith Creek Near New Market, VA   38.6935   
1   1645704      Difficult Run Above Fox Lake Near Fairfax, VA   38.8847   
2   1645762  S F Little Difficult Run Above Mouth NR Vienna...   38.9089   
3   1646000                 Difficult Run Near Great Falls, VA   38.9759   
4   1646305           Dead Run at Whann Avenue Near Mclean, VA   38.9598   
5   1649190                 Paint Branch Near College Park, MD   39.0331   
6   1649500  Northeast Branch Anacostia River at Riverdale, MD   38.9603   
7   1654000                  Accotink Creek Near Annandale, VA   38.8129   
8   1656903  Flatlick Branch Above Frog Branch at Chantilly...   38.8824   
9   1673000                    Pamunkey River Near Hanover, VA   37.7676   
10  2035000                    James River at Cartersville, VA   37.6708   
11  3353200                      Eagle Creek at Zionsville, IN   39.9464   
12  4015330                   Knife River near Two Harbors, MN   46.9469   
13  5054000                Red River of the North at Fargo, ND   46.8611   
14  5082500          Red River of the North at Grand Forks, ND   47.9269   
15  5374900                        Zumbro River at Kellogg, MN   44.3119   
16  5515500                        Kankakee River at Davis, IN   41.3896   
17  5518000                       Kankakee River at Shelby, IN   41.1828   
18  6329500                  Yellowstone River near Sidney, MT   47.6774   
19  6795500                      Shell Creek near Columbus, NE   41.5261   
20  6800500                      Elkhorn River at Waterloo, NE   41.2933   
21  6818000                   Missouri River at St. Joseph, MO   39.7533   
22  6887500                         Kansas River at Wamego, KS   39.1983   
23  6892350                         Kansas River at DeSoto, KS   38.9833   
24  6934500                      Missouri River at Hermann, MO   38.7098   
25  7144100            Little Arkansas River near Sedgwick, KS   37.8831   
26  7179750    Neosho River at Burlingame Rd. near Emporia, KS   38.4290   
27  7182250                   Cottonwood River at Plymouth, KS   38.3975   
28  7182390                Neosho River near Neosho Rapids, KS   38.3680   
29  7182510                     Neosho River at Burlington, KS   38.1945   
30  8067252                   Trinity River at Wallisville, TX   29.8124   
31  9326500       Ferron Creek (Upper Station) near Ferron, UT   39.1041   
32  9327000  Ferron Creek Below Millsite Res & Divs Near Fe...   39.0953   

    Longitude                                           geometry  \
0    -78.6428  POLYGON ((-78.63279999999999 38.7035, -78.6327...   
1    -77.3324  POLYGON ((-77.3224 38.8947, -77.3224 38.8747, ...   
2    -77.3383  POLYGON ((-77.3283 38.9189, -77.3283 38.8989, ...   
3    -77.2458  POLYGON ((-77.2358 38.9859, -77.2358 38.9659, ...   
4    -77.1757  POLYGON ((-77.1657 38.9698, -77.1657 38.9498, ...   
5    -76.9643  POLYGON ((-76.95429999999999 39.0431, -76.9542...   
6    -76.9260  POLYGON ((-76.916 38.97029999999999, -76.916 3...   
7    -77.2283  POLYGON ((-77.2183 38.8229, -77.2183 38.8029, ...   
8    -77.4319  POLYGON ((-77.42189999999999 38.89239999999999...   
9    -77.3322  POLYGON ((-77.3222 37.7776, -77.3222 37.7576, ...   
10   -78.0861  POLYGON ((-78.0761 37.6808, -78.0761 37.6608, ...   
11   -86.2603  POLYGON ((-86.2503 39.9564, -86.2503 39.9364, ...   
12   -91.7924  POLYGON ((-91.7824 46.9569, -91.7824 46.9369, ...   
13   -96.7837  POLYGON ((-96.77369999999999 46.8711, -96.7736...   
14   -97.0292  POLYGON ((-97.0192 47.9369, -97.0192 47.916900...   
15   -92.0041  POLYGON ((-91.99409999999999 44.3219, -91.9940...   
16   -86.7062  POLYGON ((-86.69619999999999 41.3996, -86.6961...   
17   -87.3403  POLYGON ((-87.33029999999999 41.1928, -87.3302...   
18  -104.1550  POLYGON ((-104.145 47.6874, -104.145 47.6674, ...   
19   -97.2817  POLYGON ((-97.2717 41.5361, -97.2717 41.5161

## 500 mb per 3 channel image on sentinel-2 including scl downsampled

In [12]:
total_tb = np.sum(img_df.cloudless_img_count)*500/1000000

In [17]:
print(f'USGS total memory needed for {np.sum(img_df.cloudless_img_count)} images: {total_tb} TB')

USGS total memory needed for 3564 images: 1.782 TB


In [18]:
#this does not include prediction

In [21]:
ana = 29*500/1000000

In [23]:
total_tb + ana

1.7965

In [4]:
### Environment setup
from pystac_client import Client
import planetary_computer as pc
import os

# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)
env_vars = !cat /content/.env

for var in env_vars:
    key, value = var.split(' = ')
    os.environ[key] = value

In [76]:
import pandas as pd
import fsspec
import numpy as np
import geopandas as gpd
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

container = 'usgs-data'

storage_options={'account_name':os.environ['CSP_ACCOUNT_NAME'],\
                 'account_key':os.environ['CSP_BLOB_KEY']}
fs = fsspec.filesystem('az', account_name=storage_options['account_name'], account_key=storage_options['account_key'])
fs_list = fs.ls(f'{container}/stations')

fluvius_storage={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY']}

In [78]:
search_term = 'csv'
fs_query = [f for f in fs_list if search_term in f]

In [79]:
for f in fs_query:
    src = f'az://{f}'
    df = pd.read_csv(src, storage_options=storage_options)
    out= f'az://{f}'
    df.to_csv(out, storage_options=fluvius_storage, index=False)